**Lucas Menezes Costa**

Dataset utilizado: [Base COFOG 2022](https://www.tesourotransparente.gov.br/ckan/dataset/despesas-por-funcao-do-governo-central-classificacao-cofog/resource/db8d7b1d-b581-4bc9-a9aa-447f0d29f301)



> A COFOG é uma classificação internacional detalhada das funções ou objetivos socioeconômicos perseguidos pelas unidades das administrações públicas por meio da aplicação dos recursos em diferentes tipos de despesa.



75991 registros

Modelo utilizado: [intfloat/multilingual-e5-large](https://huggingface.co/intfloat/multilingual-e5-large)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! pip install tqdm

In [2]:
! pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.5/93.5 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 61.0 MB/s eta 0:00:0

In [3]:
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import chromadb

Carregando o dataset

In [4]:
file_path = 'drive/MyDrive/datasets/PLN/Dados-abertos-COFOG-GC-2022.csv'
data = pd.read_csv(file_path, encoding='latin1', delimiter=';')
data.head()

<ipython-input-4-7fc5985b9398>:2: DtypeWarning: Columns (1,2,6) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path, encoding='latin1', delimiter=';')


,ANO,AJUSTE METODOLOGICO,NATUREZA DESPESA DETALHADA,NATUREZA DESPESA DETALHADA DESCRICAO,ACAO GOVERNO,ACAO GOVERNO DESCRICAO,UNIDADE ORCAMENTARIA,UNIDADE ORCAMENTARIA DESCRICAO,CLASSIFICACAO ECONOMICA GFS,CLASSIFICACAO ECONOMICA GFS DESCRICAO,CLASSIFICACAO COFOG,CLASSIFICACAO COFOG DESCRICAO,VALOR (R$)
0,2022,NaN,31900101,PROVENTOS - PESSOAL CIVIL,00S6,BENEFICIO ESPECIAL E DEMAIS COMPLEMENTACOES DE...,15106,TRIBUNAL REGIONAL DO TRABALHO DA 5A. REGIAO,273,GASTO - BENEFICIOS PREVIDENCIARIOS E ASSISTEN...,7102,PROTECAO SOCIAL - TERCEIRA IDADE,"590.097,77"
1,2022,NaN,31900101,PROVENTOS - PESSOAL CIVIL,00S6,BENEFICIO ESPECIAL E DEMAIS COMPLEMENTACOES DE...,26402,"INST.FED.DE EDUC.,CIENC.E TEC.DE ALAGOAS",273,GASTO - BENEFICIOS PREVIDENCIARIOS E ASSISTEN...,7102,PROTECAO SOCIAL - TERCEIRA IDADE,"1.000,00"
2,2022,NaN,31900101,PROVENTOS - PESSOAL CIVIL,0181,APOSENTADORIAS E PENSOES CIVIS DA UNIAO,1101,CAMARA DOS DEPUTADOS,273,GASTO - BENEFICIOS PREVIDENCIARIOS E ASSISTEN...,7102,PROTECAO SOCIAL - TERCEIRA IDADE,"204.007.538,67"
3,2022,NaN,31900101,PROVENTOS - PESSOAL CIVIL,0181,APOSENTADORIAS E PENSOES CIVIS DA UNIAO,2101,SENADO FEDERAL,273,GASTO - BENEFICIOS PREVIDENCIARIOS E ASSISTEN...,7102,PROTECAO SOCIAL - TERCEIRA IDADE,"1.116.992.750,40"
4,2022,NaN,31900101,PROVENTOS - PESSOAL CIVIL,0181,APOSENTADORIAS E PENSOES CIVIS DA UNIAO,3101,TRIBUNAL DE CONTAS DA UNIAO,273,GASTO - BENEFICIOS PREVIDENCIARIOS E ASSISTEN...,7102,PROTECAO SOCIAL - TERCEIRA IDADE,"168.474.242,60"


Essa função tem como objetivo retornar a média dos hidden states válidos. Primeiro é utilizado a attention_mask para transformar os hidden states irrelevantes (que representam tokens com padding para preencher o tamanho do da sequencia). Depois, é feita a soma de todos os hidden states e dividido pela quantidade de tokens importantes. (não sei se "importantes" seria a palavra mais adequada)

In [5]:
def average_pool(last_hidden_states, attention_mask):
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]

Inicializar o modelo e tokenizer

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [7]:
tokenizer = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-large')
model = AutoModel.from_pretrained('intfloat/multilingual-e5-large').to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

Função para gerar embeddings para uma coluna em lotes.

In [8]:
def get_embeddings(texts, batch_size=1024):
    embeddings = []
    for i in tqdm(range(0, len(texts), batch_size), desc="Processando Embeddings"):
        batch_texts = texts[i:i+batch_size]
        # Aqui eu faço o tokenizer dos textos em cada batch
        # Defino o tamanho máximo de 512 tokens e com padding ativo
        batch_dict = tokenizer(batch_texts, max_length=512, padding=True, truncation=True, return_tensors='pt').to(device)
        # O no_grad serve para que não seja feito o gradiente
        with torch.no_grad():
            # Aqui eu passo os tokens pelo modelo para que ele capture os significados semânticos,
            # que são representados como hidden state que expliquei na parte do average pooling, e assim gerado o embedding.
            outputs = model(**batch_dict)
            # Agora que foi gerado os embedding, posso passar os hidden states na função juntamente com as attention mask
            # gerando assim a média para representar vetorialmente o texto
            batch_embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])
        # Por fim, faço a normalização com L2 para que o tamanho do vetor seja menor e padronizado
        embeddings.append(F.normalize(batch_embeddings, p=2, dim=1).cpu().numpy())
    return np.vstack(embeddings)

Diretório onde armazeno os embeddings

In [9]:
output_dir = 'drive/MyDrive/datasets/PLN/output_embeddings/'
os.makedirs(output_dir, exist_ok=True)

Aqui eu passo por cada coluna do meu dataset, gero uma lista com os textos dessa coluna, e executo o get_embeddings, depois transformo em dict e salvo esses embedding no meu drive.



In [12]:
def clean_column_name(name):
    clean_name = name.strip().replace(' ', '_').replace('.', '_').replace('-', '_')
    clean_name = ''.join(c for c in clean_name if c.isalnum() or c in ['_', '-'])
    return clean_name[:63]

In [10]:
embeddings_dict = {}
for column in tqdm(data.columns, desc="Processando colunas"):
    texts = data[column].astype(str).tolist()
    embeddings = get_embeddings(texts)
    embeddings_dict[column] = embeddings
    np.save(os.path.join(output_dir, f'{clean_column_name(column)}_embeddings.npy'), embeddings)

Processando colunas:   0%|          | 0/13 [00:00<?, ?it/s]

Não é a correta
Não é a correta
Não é a correta
Não é a correta
Não é a correta
Não é a correta
Não é a correta
É a correta



Processando colunas: 100%|██████████| 13/13 [05:39<00:00, 26.09s/it]

Não é a correta
Não é a correta
Não é a correta
Não é a correta
Não é a correta


Inicializando o cliente ChromaDB

In [11]:
client = chromadb.Client()

Função para limpar e validar nomes de coleções (principalmente para remover espaços entre palavras)

Função para verificar se uma coleção já existe

In [13]:
def collection_exists(client, name):
    try:
        client.get_collection(name=name)
        return True
    except ValueError:
        return False

Separando apenas 2 colunas para ser mais rápido

In [14]:
# columns_to_test = ['NATUREZA DESPESA DETALHADA DESCRICAO', 'ACAO GOVERNO DESCRICAO']
columns_to_test = ['UNIDADE ORCAMENTARIA DESCRICAO']

Nessa parte, armazeno os embeddings salvos no drive em coleções no chromadb

In [17]:
collections = {}
embeddings_dict = {}

try:
    for column in columns_to_test:
        clean_name = clean_column_name(column)
        file_path = os.path.join(output_dir, f"{clean_name}_embeddings.npy")

        # Carrega os embeddings do arquivo .npy
        embeddings = np.load(file_path)
        embeddings_dict[clean_name] = embeddings

        # Certifica-se de que a coleção existe, caso contrário, cria
        if not collection_exists(client, clean_name):
            collection = client.create_collection(name=clean_name)
        else:
            collection = client.get_collection(name=clean_name)

        # Aqui eu pego os textos correspondentes da coluna original para passar juntamente com seu embedding
        # e termos uma representação visual durante os testes
        texts = data[column].astype(str).tolist()

        # E agora faz o armazenamento dos embeddings e documentos no chromadb
        for i, embedding in enumerate(tqdm(embeddings, desc=f"Armazenando embeddings para {column}")):
            try:
                collection.add(
                    ids=[f"{clean_name}_{i}"],
                    embeddings=[embedding.tolist()],
                    metadatas=[{"index": i}],
                    documents=[texts[i]]
                )
            except Exception as e:
                print(f"Erro ao adicionar embedding {i} para {column}: {e}")

        collections[clean_name] = collection
except Exception as e:
    print(f"Ocorreu um erro: {e}")

Armazenando embeddings para UNIDADE ORCAMENTARIA DESCRICAO: 100%|██████████| 75991/75991 [03:42<00:00, 342.24it/s]


In [18]:
print(collections)

{'UNIDADE_ORCAMENTARIA_DESCRICAO': Collection(id=dfb07f7b-5ab6-418d-83d1-985b99b9508a, name=UNIDADE_ORCAMENTARIA_DESCRICAO)}


Exportar embedding

In [26]:
# Testar a similaridade usando um dos embeddings carregados

# query_string = "Guerra"
# query_string = "roupa"
query_string = "conselho"
# query_string = "governo"
# query_string = "medida"

# Tokenizando a string de consulta
query_dict = tokenizer(query_string, max_length=512, padding=True, truncation=True, return_tensors='pt')
query_dict = {key: value.to(device) for key, value in query_dict.items()}

# Aqui eu passo os tokens pelo modelo que passei lá na geração para toda a tabela
with torch.no_grad():
    outputs = model(**query_dict)
    query_vector = average_pool(outputs.last_hidden_state, query_dict['attention_mask'])

# Fazendo também a normalização
query_vector = F.normalize(query_vector, p=2, dim=1).cpu().numpy()

# Removendo a dimensão extra (flattening)
query_vector = query_vector.flatten()

results = collections['UNIDADE_ORCAMENTARIA_DESCRICAO'].query(query_embeddings=[query_vector.tolist()], n_results=1000)

print(results['documents'])

[['CAMARA DOS DEPUTADOS', 'CAMARA DOS DEPUTADOS', 'CAMARA DOS DEPUTADOS', 'CAMARA DOS DEPUTADOS', 'CAMARA DOS DEPUTADOS', 'CAMARA DOS DEPUTADOS', 'CAMARA DOS DEPUTADOS', 'CAMARA DOS DEPUTADOS', 'CAMARA DOS DEPUTADOS', 'CAMARA DOS DEPUTADOS', 'CAMARA DOS DEPUTADOS', 'CAMARA DOS DEPUTADOS', 'CAMARA DOS DEPUTADOS', 'CAMARA DOS DEPUTADOS', 'CAMARA DOS DEPUTADOS', 'CAMARA DOS DEPUTADOS', 'CAMARA DOS DEPUTADOS', 'CAMARA DOS DEPUTADOS', 'CAMARA DOS DEPUTADOS', 'CAMARA DOS DEPUTADOS', 'CAMARA DOS DEPUTADOS', 'CAMARA DOS DEPUTADOS', 'CAMARA DOS DEPUTADOS', 'CAMARA DOS DEPUTADOS', 'CAMARA DOS DEPUTADOS', 'CAMARA DOS DEPUTADOS', 'CAMARA DOS DEPUTADOS', 'CAMARA DOS DEPUTADOS', 'CAMARA DOS DEPUTADOS', 'CAMARA DOS DEPUTADOS', 'CAMARA DOS DEPUTADOS', 'CAMARA DOS DEPUTADOS', 'CAMARA DOS DEPUTADOS', 'CAMARA DOS DEPUTADOS', 'CAMARA DOS DEPUTADOS', 'CAMARA DOS DEPUTADOS', 'CAMARA DOS DEPUTADOS', 'CAMARA DOS DEPUTADOS', 'CAMARA DOS DEPUTADOS', 'CAMARA DOS DEPUTADOS', 'CAMARA DOS DEPUTADOS', 'CAMARA DOS DE